<a href="https://colab.research.google.com/github/vngeno/E-COMMERCE-RECOMMENDATION-SYSTEM-/blob/Hybrid-Based-Filtering/Hybrid_Recommenders_for_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install wandb
!pip install surprise
!pip install fast_ml
!pip install rake_nltk

     |████████████████████████████████| 11.8 MB 51 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619403 sha256=a4f61a3e397ab73bc1058772c898418d044a39101d3645ba9b355c7a83a98c4d
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise
     |████████████████████████████████| 42 kB 434 kB/s 
     |████████████████████████████████| 1.5 MB 5.9 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


## Load the Libraries and load the data

In [ ]:
#manipulating data
import pandas as pd
import numpy as np

from rake_nltk import Rake
#Sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA

In [ ]:
#load the data
df = pd.read_csv("/content/drive/Shareddrives/Team Stars/Data/df_clean.csv",index_col=0)
df.head(2)

,id,name,brand,primarycategories,manufacturernumber,reviews_dorecommend,reviews_numhelpful,reviews_rating,reviews_text,reviews_title,reviews_username
0,AVpgNzjwLJeJML43Kpxn,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang
1,AVpgNzjwLJeJML43Kpxn,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG


In [ ]:
df=df.rename(columns={'reviews_username':'username','reviews_rating':'ratings','reviews_text':'desc'})
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df.username=le.fit_transform(df.username)
df.id=le.fit_transform(df.id)
df.head()

,id,name,brand,primarycategories,manufacturernumber,reviews_dorecommend,reviews_numhelpful,ratings,desc,reviews_title,username
0,21,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,8417
1,21,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,5201
2,21,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,6994
3,21,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,5,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,8769
4,21,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,5,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,8713


## Feature Selection

In [ ]:
data=df[['id','username','name','brand','ratings','desc']]
data=data.rename(columns={'username':'userid','name':'productname','id':'productid'})
data.head()

,productid,userid,productname,brand,ratings,desc
0,21,8417,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,3,I order 3 of them and one of the item is bad q...
1,21,5201,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,4,Bulk is always the less expensive way to go fo...
2,21,6994,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,5,Well they are not Duracell but for the price i...
3,21,8769,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,5,Seem to work as well as name brand batteries a...
4,21,8713,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,5,These batteries are very long lasting the pric...


# Content based

## Text Preprocessing

In [ ]:
import re
#Text process
data['brand'] = [re.sub(r'[^\w\s]', '', t) for t in data['brand']]
data['desc'] = [re.sub(r'[^\w\s]', '', t) for t in data['desc']]

data['brand'] = [t.lower() for t in data['brand']]
data['desc'] = [t.lower() for t in data['desc']]
data.head()

,productid,userid,productname,brand,ratings,desc
0,21,8417,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,3,i order 3 of them and one of the item is bad q...
1,21,5201,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,4,bulk is always the less expensive way to go fo...
2,21,6994,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,5,well they are not duracell but for the price i...
3,21,8769,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,5,seem to work as well as name brand batteries a...
4,21,8713,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,5,these batteries are very long lasting the pric...


In [ ]:
for index, row in data.iterrows():
  row['desc']=[x.lower().replace(' ','')for x in row['desc']]
  row['desc'] = ''.join(row['desc']).lower()
  row['brand'] = ''.join(row['brand']).lower()
data.head()

,productid,userid,productname,brand,ratings,desc
0,21,8417,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,3,i order 3 of them and one of the item is bad q...
1,21,5201,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,4,bulk is always the less expensive way to go fo...
2,21,6994,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,5,well they are not duracell but for the price i...
3,21,8769,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,5,seem to work as well as name brand batteries a...
4,21,8713,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,5,these batteries are very long lasting the pric...


In [ ]:
 import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data['keywords']=""
for index,row in data.iterrows():
  desc=row['desc']
  rake_nltk_var = Rake()
  rake_nltk_var.extract_keywords_from_text(desc)
  keyword_extracted = rake_nltk_var.get_word_degrees()
  row['keywords']=list(keyword_extracted.keys())
  #Intialize rake
  # r=Rake()
  # #extract keywords
  # r.extract_keywords_from_text(desc)
  # #getting the dict with key words and their scores
  # row['keywords'] = r.get_ranked_phrases()

  #assigning the key words to new column
  #  = list(key_words_dict_scores.keys())

#dropping the desc column
#data.drop(columns=['desc'],inplace=True)
# data.keywords.unique()

print(data.keywords)

0         
1         
2         
3         
4         
        ..
28327     
28328     
28329     
28330     
28331     
Name: keywords, Length: 28332, dtype: object


In [ ]:
data.head()

,productid,userid,productname,brand,ratings,desc,keywords
0,21,8417,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,3,i order 3 of them and one of the item is bad q...,
1,21,5201,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,4,bulk is always the less expensive way to go fo...,
2,21,6994,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,5,well they are not duracell but for the price i...,
3,21,8769,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,5,seem to work as well as name brand batteries a...,
4,21,8713,AmazonBasics AAA Performance Alkaline Batterie...,amazonbasics,5,these batteries are very long lasting the pric...,


# Collaborative filtering

In [ ]:
# pivot table
rmat = data.pivot_table(
    columns = 'productid',
    index = 'userid',
    values = 'ratings'
)
rmat.fillna(0)

productid,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
userid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Fit into the title
#Lets vectorize all these reviews. Iinitialize vectorizer
vect = CountVectorizer(analyzer = 'word', ngram_range = (1,2), stop_words = 'english', min_df = 0.002) 
#min_df = rare words, max_df = most used words
#ngram_range = (1,2) - if used more than  1(value), lots of features or noise
vect.fit(data['desc'])
title_matrix = vect.transform(data['desc'])
title_matrix.shape

(28332, 1095)

In [ ]:
#Lets find features
features = vect.get_feature_names()
#performing cosine similarity
cosine_sim = cosine_similarity(title_matrix, title_matrix)
cosine_sim.shape

(28332, 28332)

In [ ]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 5))
dff = Dataset.load_from_df(data[['userid', 'productid', 'ratings']], reader)

In [ ]:
from surprise import SVD
from surprise.model_selection import cross_validate

svd = SVD(verbose=True, n_epochs=10)
# cross_validate(svd, dff, measures=['RMSE', 'MAE'], cv=3, verbose=True)

In [ ]:
# split data into train test
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)
trainset = dff.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [ ]:
# trainset

In [ ]:
# svd.predict(uid=10, iid=100)

In [ ]:
# hybrid
def hybrid(userid, productid, svd_model,n_recs ):
    
    # sort similarity values in decreasing order and take top 50 results
    sim = list(enumerate(cosine_sim[int(productid)]))
    sim = sorted(sim, key=lambda x: x[1], reverse=True)
    sim = sim[1:50]
    
    # get product metadata
    product_idx = [i[0] for i in sim]
    products = data.iloc[product_idx][['productid', 'ratings']]
    
    # predict using the svd_model
    products['est'] = products.apply(lambda x: svd_model.predict(userid, x['productid'], x['ratings']).est, axis = 1)
    
    # sort predictions in decreasing order and return top n_recs
    products = products.sort_values('est', ascending=False)
    return products.head(n_recs)
  
# generate recommendations
hybrid(244,4539,svd,5)

,productid,ratings,est
10437,7,4,4.367562
9179,7,5,4.367562
12084,7,5,4.367562
12008,7,5,4.367562
11597,7,5,4.367562
